In [15]:
import random
import pandas as pd
import numpy as np
import re
import pickle
import ir_datasets
import time
import heapq
from rerankers import Reranker
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# **Split em conjuntos**

Datasets de treino e teste

In [3]:
random.seed(42)

def load_dataset(input_file):
    with open(input_file, 'rb') as f:
        return pickle.load(f)

data_set = "../subset_msmarco_train_0/subset_msmarco_train_0.01_99.pkl"

data = load_dataset(data_set)
queries = data["queries"]
documents = data["docs"]

# Split the queries (queries is a dictionary of {query_id: query_object})
query_ids = list(queries.keys())  # List of query IDs

# Shuffle query IDs to ensure a random split
random.shuffle(query_ids)

# Split into 80% for training, 20% for validation
split_ratio = 0.8
train_query_ids = query_ids[:int(len(query_ids) * split_ratio)]
test_query_ids = query_ids[int(len(query_ids) * split_ratio):]

train_queries = {qid: queries[qid] for qid in train_query_ids}
test_queries = {qid: queries[qid] for qid in test_query_ids}

# **Tratamento**

In [4]:
# Dicionário agrupa os documentos relevantes por consulta (query)
relevant_docs = dict()

for qrel in data["qrels"]:
    relevant_docs[qrel.query_id] = relevant_docs.get(qrel.query_id, []) + [qrel.doc_id]

# e.g.: relevant_docs = {'query_id': ['doc_id_1', 'doc_id_12']}

In [5]:
# Identifica os documentos relevantes no conjunto de treinamento 
# removendo duplicatas
train_docs = set()
for qid in train_query_ids:
    train_docs.update(relevant_docs[qid])

The **mean reciprocal rank** is a statistic measure for evaluating any process that produces a list of possible responses to a sample of queries, ordered by probability of correctness. The reciprocal rank of a query response is the multiplicative inverse of the rank of the first correct answer: $1$ for first place, $\frac{1}{2}$ for second place, $\frac{1}{3}$ for third place and so on. The mean reciprocal rank is the average of the reciprocal ranks of results for a sample of queries Q:

$$
MRR =
\frac{1}{\|Q\|} = \sum_{i=1}^{\|Q\|} \frac{1}{rank_i}
$$

where ${\displaystyle {\text{rank}}_{i}}$ refers to the rank position of the first relevant document for the i-th query.

The reciprocal value of the mean reciprocal rank corresponds to the harmonic mean of the ranks.

In [6]:
def calculateMRR(query_id, ranking, relevant_docs, top_k=None):
    """
    Calcula o MRR (Mean Reciprocal Rank) para uma consulta.
    
    :param query_id: ID da consulta atual.
    :param ranking: Lista de documentos recuperados ordenados (tuplas com o ID e a pontuação).
    :param relevant_docs: Dicionário com os documentos relevantes para cada consulta.
    :param top_k: Número de documentos a considerar. Se None, considera todos os documentos.
    
    :return: MRR para a consulta.
    """
    
    if top_k is not None:
        ranking = ranking[:top_k]
    
    for i, doc in enumerate(ranking):
        doc_id = doc[0]  # O primeiro elemento de cada tupla é o ID do documento
        if doc_id in relevant_docs[query_id]:
            return 1 / (i + 1)  # Retorna o recíproco da posição do primeiro documento relevante
    
    return 0  # Se não houver documento relevante nos primeiros K

def calculate_recall_at_k(relevant_docs, doc_ranking, top_k=100):
    """
    Calcular o Recall@K (com K=100) para uma consulta.
    """
    
    retrieved_relevant_docs = sum(1 for doc_id in doc_ranking[:top_k] if doc_id in relevant_docs)
    
    if relevant_docs:
        return retrieved_relevant_docs / len(relevant_docs)
    else:
        return 0

In [7]:
def getRelevantDocTexts(query_id):
    """
    Retorna os textos dos documentos relevantes para uma consulta.
    
    :param query_id: ID da query.
    
    :return: Lista de textos dos documentos relevantes.
    """
    
    relevant_doc_texts = []
    for doc_id in relevant_docs[query_id]:
        relevant_doc_texts.append(data["docs"][doc_id].text)

    return relevant_doc_texts

In [8]:
def preprocess(text, remove_punctuation = True, lowercase = True):
    """
    Função para preprocessamento de texto: remove pontuação, converte para minúsculas e divide em tokens.
    
    :param text: Texto a ser processado.
    :remove_punctuation bool: Para True remove pontuação
    :lowercase bool: Para True coloca em lowercase

    :return: Lista de tokens do texto.
    """

    # [^\w\s] corresponde a qualquer caractere que não seja uma letra, número, underscore ou espaço em branco
    # mantendo apenas letras, números e espaços

    # Caso 1
    if remove_punctuation:
        text = re.sub(r"[^\w\s]", "", text)
    
    # Caso 2
    if lowercase:
        text = text.lower()

    return text.split()

# **Modelo baseline**

## **BM25**

O BM25Okapi é uma implementação do BM25, um modelo clássico utilizado em sistemas de recuperação de informações (como motores de busca) para avaliar a relevância de documentos em relação a uma consulta (query).

O BM25 é uma fórmula de pontuação de relevância que calcula o quão bem um documento corresponde a uma consulta com base nas palavras que o documento contém.

Ele é um modelo probabilístico de recuperação de informações, baseado na ideia de que a relevância de um documento para uma consulta depende da frequência das palavras que aparecem no documento e na consulta, mas com uma diminuição das contribuições das palavras que ocorrem com frequência excessiva.

In [ ]:
def baseline_model(query_id, bm25, remove_punctuation, lowercase, top_k=100):
    """
        Usa o BM25 para recuperar o ranking de documentos relevantes para uma consulta.
    """

    query = queries[query_id].text
    tokenized_query = preprocess(query, remove_punctuation, lowercase)

    doc_scores = bm25.get_scores(tokenized_query)
    
    # Manter os top_k documentos mais relevantes usando um heap
    doc_ranking = heapq.nlargest(top_k, zip(data["docs"].keys(), doc_scores), key=lambda x: x[1])

    return doc_ranking


def evaluate_model(query_ids, relevant_docs, remove_punctuation, lowercase, documents, top_k=100):
    """
    Avalia o modelo em termos de MRR@K, Recall@100, tempo de execução e tempo máximo de execução.

    :param query_ids: IDs das consultas a serem avaliadas.
    :param relevant_docs: Dicionário com documentos relevantes para cada consulta.
    :complete
    :param documents: Dicionário com documentos do modelo.
    :param top_k: Número de documentos a considerar para Recall@100 e MRR@10.
    
    :return: Tuple com as métricas calculadas: (average_mrr, average_time, max_execution_time, average_recall_100)
    """
    
    total_mrr = 0
    total_recall_100 = 0
    total_execution = 0
    max_execution = 0

    tokenized_corpus = [preprocess(doc.text, remove_punctuation, lowercase) for doc in data["docs"].values()]
    bm25 = BM25Okapi(tokenized_corpus)

    # Avaliar cada consulta
    for query_id in query_ids:
        start_time = time.time()

        # Recupera o ranking de documentos para a consulta
        doc_ranking = baseline_model(query_id, bm25, remove_punctuation, lowercase)
        
        end_time = time.time()
        execution_time = end_time - start_time
        total_execution += execution_time

        # Atualiza o tempo máximo de execução
        if execution_time > max_execution:
            max_execution = execution_time

        # Calcular MRR@10
        mrr_for_query = calculateMRR(query_id, doc_ranking, relevant_docs, top_k=10)
        total_mrr += mrr_for_query

        # Calcular Recall@100
        recall_for_query = calculate_recall_at_k(relevant_docs[query_id], doc_ranking, top_k=100)
        total_recall_100 += recall_for_query

    # Calcular as médias
    average_mrr = total_mrr / len(query_ids)
    average_time = total_execution / len(query_ids)
    average_recall_100 = total_recall_100 / len(query_ids)

    return average_mrr, average_time, max_execution, average_recall_100

In [ ]:
results_data = []

Dataset de teste

In [ ]:
average_mrr, average_time, max_execution, average_recall_100 = evaluate_model(
    test_query_ids, 
    relevant_docs, 
    remove_punctuation=True, 
    lowercase=True, 
    documents=data["docs"], 
    top_k=10
)

results_data.append(["Test", len(test_query_ids), average_mrr, average_recall_100, average_time, max_execution])

print("Resultados para o conjunto de teste:")
print(f"MRR@10 médio: {average_mrr}")
print(f"Tempo médio de execução por consulta: {average_time}")
print(f"Tempo máximo de execução por consulta: {max_execution}")
print(f"Recall@100 médio: {average_recall_100}")

Resultados para o conjunto de teste:
MRR@10 médio: 0.480087945087945
Tempo médio de execução por consulta: 0.41367159963728073
Tempo máximo de execução por consulta: 1.2225031852722168
Recall@100 médio: 0.0


Dataset de treino

In [19]:
average_mrr, average_time, max_execution, average_recall_100 = evaluate_model(
    train_query_ids, 
    relevant_docs, 
    remove_punctuation=True, 
    lowercase=True, 
    documents=data["docs"], 
    top_k=10
)

results_data.append(["Test", len(test_query_ids), average_mrr, average_recall_100, average_time, max_execution])

print("Resultados para o conjunto de treinamento:")
print(f"MRR@10 médio: {average_mrr}")
print(f"Tempo médio de execução por consulta: {average_time}")
print(f"Tempo máximo de execução por consulta: {max_execution}")
print(f"Recall@100 médio: {average_recall_100}")

Resultados para o conjunto de treinamento:
MRR@10 médio: 0.4421159604034162
Tempo médio de execução por consulta: 0.42464258739664235
Tempo máximo de execução por consulta: 1.8286347389221191
Recall@100 médio: 0.0


Dataset completo

In [ ]:
average_mrr, average_time, max_execution, average_recall_100 = evaluate_model(
    query_ids, 
    relevant_docs, 
    remove_punctuation=True, 
    lowercase=True, 
    documents=data["docs"], 
    top_k=10
)

results_data.append(["Full", len(test_query_ids), average_mrr, average_recall_100, average_time, max_execution])

print("Resultados para o conjunto completo:")
print(f"MRR@10 médio: {average_mrr}")
print(f"Tempo médio de execução por consulta: {average_time}")
print(f"Tempo máximo de execução por consulta: {max_execution}")
print(f"Recall@100 médio: {average_recall_100}")

In [ ]:
bm_25_results = pd.DataFrame(results_data, columns=["Dataset", "Size", "MRR@10", "Recall@100", "Average query runtime (sec)", "Maximum query runtime (sec)"])
bm_25_results

Resultado recente:
- Com processamento dos documentos
- Com processamento das queries

Obs.: O processamento inclui remover qualquer caractere que não seja uma letra, número, underscore ou espaço em branco; e deixar em lowercase.

In [94]:
print("==> Estudo de escalabilidade:")
bm_25_results

==> Estudo de escalabilidade:


,Dataset,Size,MRR,Runtime (sec)
0,Test,555,0.488720,298.350545
1,Train,2216,0.449240,1220.195126
2,Full,2771,0.457147,1467.348227


In [ ]:
bm_25_results.to_csv("../results/bm_25_results.txt")

Resultado anterior:
- Com processamento dos documentos
- Sem processamento das queries (apenas lowercase)

In [ ]:
print("==> Estudo de escalabilidade:")
bm_25_results

,Dataset,Size,MRR,Runtime
0,Test,555,0.474292,257.141707
1,Train,2216,0.428867,1072.796115
2,Full,2771,0.437965,1340.438707


Ablação na etapa de tratamento dos dados:

In [ ]:
# DataFrame para armazenar os resultados
bm_25_processing_results = pd.DataFrame(columns=[
    "Remove punctuation", 
    "Lowercase", 
    "MRR@10", 
    "Recall@100", 
    "Average query runtime (sec)", 
    "Maximum query runtime (sec)"
])

index = 0

# Teste de ablação para as combinações de remove_punctuation e lowercase
for remove_punctuation in [True, False]:
    for lowercase in [True, False]:
        # Variáveis para armazenar os resultados
        average_mrr = 0
        total_recall_100 = 0
        total_execution = 0
        max_execution = 0

        # Tempo total de execução para o conjunto de teste
        start_time = time.time()

        # Avaliar o modelo para cada consulta no conjunto de teste
        for query_id in test_query_ids:
            # Chama a função de avaliação com os parâmetros de pre-processamento
            mrr, avg_time, max_time, recall_100 = evaluate_model(
                [query_id],  # Passa a consulta individual
                relevant_docs, 
                remove_punctuation=remove_punctuation, 
                lowercase=lowercase, 
                documents=data["docs"], 
                top_k=10  # Para MRR@10
            )
            
            # Acumula as métricas
            average_mrr += mrr
            total_recall_100 += recall_100
            total_execution += avg_time
            if max_time > max_execution:
                max_execution = max_time

        # Calcular médias
        average_mrr /= len(test_query_ids)
        average_recall_100 = total_recall_100 / len(test_query_ids)
        average_time = total_execution / len(test_query_ids)

        # Calcula o tempo total de execução
        end_time = time.time()
        execution_time = end_time - start_time

        # Armazenar os resultados no DataFrame
        bm_25_processing_results.loc[index] = [
            remove_punctuation, 
            lowercase, 
            average_mrr, 
            average_recall_100, 
            average_time, 
            max_execution
        ]

        index += 1

print(bm_25_processing_results)

In [99]:
print("==> Estudo do desempenho do tratamento de dados:")
bm_25_processing_results

==> Estudo do desempenho do tratamento de dados:


,Remove punctuation,Lowercase,MRR,Runtime (sec)
0,True,True,0.488720,297.235475
1,True,False,0.488860,289.510962
2,False,True,0.474292,297.298308
3,False,False,0.474432,288.388891


In [ ]:
bm_25_processing_results.to_csv("../results/bm_25_processing_results.txt")

## **TF-IDF (Term Frequency-Inverse Document Frequency)**

In [16]:
def initialize_tfidf(documents):
    """
        Inicializa o TF-IDF e vetoriza todos os documentos uma vez
    """

    # Extrair o texto dos documentos
    doc_texts = [doc.text for doc in documents.values()]
    
    # Inicializar o TfidfVectorizer
    vectorizer = TfidfVectorizer()
    
    # Vetorizar todos os documentos
    tfidf_matrix = vectorizer.fit_transform(doc_texts)
    
    return vectorizer, tfidf_matrix


def retrieve_relevant_documents(query, vectorizer, tfidf_matrix, documents, top_k=10):
    """
        Recupera o documento mais relevante para uma consulta
    """
    
    # Vetorizar a consulta
    query_tfidf = vectorizer.transform([query])

    # Calcular a similaridade de cosseno entre a consulta e os documentos
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)

    if top_k == 1:
        # Encontrar o índice do documento mais relevante
        most_relevant_document_index = cosine_similarities.argmax()

        # Recuperar o ID do documento mais relevante
        doc_ranking = [list(documents.keys())[most_relevant_document_index]]
        
    else:
        # Usar heap para obter os top_k documentos mais relevantes
        doc_ranking = heapq.nlargest(top_k, zip(documents.keys(), cosine_similarities[0]), key=lambda x: x[1])
        doc_ranking = [doc_id for doc_id, score in doc_ranking]

    return doc_ranking

In [13]:
def tf_idf_model(query_ids, documents, relevant_docs):

    vectorizer, tfidf_matrix = initialize_tfidf(documents)

    total_mrr = 0
    total_recall_100 = 0
    max_execution = 0
    total_execution = 0
    top_k = 100  # Considera os 100 primeiros documentos

    for query_id in query_ids:
        start_time = time.time()

        query = queries[query_id].text

        # Recuperar os 10 documentos mais relevantes
        doc_ranking = retrieve_relevant_documents(query, vectorizer, tfidf_matrix, documents, top_k)
        
        end_time = time.time()
        execution_time = end_time - start_time
        total_execution += execution_time

        # Calcular o tempo máximo de execução
        if execution_time > max_execution:
            max_execution = execution_time

        # Calcular MRR@10 (ou outro valor de K)
        mrr_for_query = calculateMRR(query_id, doc_ranking, relevant_docs, top_k)
        total_mrr += mrr_for_query

        # Calcular Recall@100
        recall_for_query = calculate_recall_at_k(relevant_docs[query_id], doc_ranking, top_k)
        total_recall_100 += recall_for_query

    # Calcular médias
    average_mrr = total_mrr / len(query_ids)
    average_time = total_execution / len(query_ids)
    average_recall_100 = total_recall_100 / len(query_ids)

    return average_mrr, average_time, max_execution, average_recall_100

In [ ]:
average_mrr, average_time, max_execution, average_recall_100 = tf_idf_model(query_ids, documents, relevant_docs)

results_data = {
    "Average MRR@10": [average_mrr],
    "Average Query Runtime (sec)": [average_time],
    "Maximum Query Runtime (sec)": [max_execution],
    "Average Recall@100": [average_recall_100]
}

results_df = pd.DataFrame(results_data)

print(f"Average MRR@10: {average_mrr}")
print(f"Average Query Runtime (sec): {average_time}")
print(f"Maximum Query Runtime (sec): {max_execution}")
print(f"Average Recall@100: {average_recall_100}")

In [ ]:
results_df.to_csv("../results/tf_idf_processing_results.txt", index=False, sep="\t")

# **Modelo Reranker**

In [ ]:
ranker = Reranker("cross-encoder", device='cuda')
tokenized_corpus = [preprocess(doc.text, remove_punctuation, lowercase) for doc in data["docs"].values()]
bm25 = BM25Okapi(tokenized_corpus)

def model(query_id, remove_punctuation, lowercase):
    query = queries[query_id].text
    tokenized_query = preprocess(query, remove_punctuation, lowercase)
    doc_scores = bm25.get_scores(tokenized_query)
    doc_ranking = sorted(zip(data["docs"].keys(), doc_scores), key=lambda x: x[1], reverse=True)
    top_10 = doc_ranking[:10]
    top_10_ids = [doc_id for doc_id, score in top_10]
    top_10_texts = [data["docs"][doc_id].text for doc_id in top_10_ids]
    reranked = ranker.rank(query=query, docs=top_10_texts, doc_ids=top_10_ids)
    doc_ids = [result.doc_id for result in reranked]
    scores = [result.score for result in reranked]
    doc_ranking = list(zip(doc_ids, scores))

    return doc_ranking

Loading default cross-encoder model for language en
If your model is NOT intended to be ran as a one-label cross-encoder, please reload it and specify the model_type! Otherwise, you may ignore this warning. You may specify `model_type='cross-encoder'` to suppress this warning in the future.
Default Model: mixedbread-ai/mxbai-rerank-base-v1
Loading TransformerRanker model mixedbread-ai/mxbai-rerank-base-v1 (this message can be suppressed by setting verbose=0)
No dtype set
Using dtype torch.float32
Loaded model mixedbread-ai/mxbai-rerank-base-v1
Using device cuda.
Using dtype torch.float32.


### Inferência

In [78]:
reranker_results = pd.DataFrame(columns=("Dataset", "Size", "MRR", "Runtime"))

In [ ]:
average_mrr2 = 0

start_time = time.time()
for query_id in test_query_ids:
    doc_ranking = model(query_id, True, True)
    average_mrr2 += calculateMRR(query_id, doc_ranking, relevant_docs)
    
average_mrr2 /= len(test_query_ids)
end_time = time.time()

execution_time = end_time - start_time

reranker_results.loc[0] = ["Test", len(test_query_ids), average_mrr2, execution_time]

Resultado recente:
- Com processamento dos documentos
- Com processamento das queries

Obs.: O processamento inclui remover qualquer caractere que não seja uma letra, número, underscore ou espaço em branco; e deixar em lowercase.

In [97]:
reranker_results

,Dataset,Size,MRR,Runtime
0,Test,555,0.593363,312.731814


Resultado anterior:
- Com processamento dos documentos
- Sem processamento das queries (apenas lowercase)

In [75]:
reranker_results

,Dataset,Size,MRR,Runtime
0,Test,555,0.582853,315.65561


In [ ]:
reranker_results.to_csv("../results/reranker_results.txt")